# Lesson 3: Enable Logging

### Import all needed packages

In [1]:
import boto3
import json
import os

In [13]:
role_arn = "arn:aws:iam::165819210796:role/serverless-llm-apps-bedrock-role"
bucket_name = "learning-apps-bucket"
project_name = "serverless-llm-apps-amazon-bedrock"
log_group_name = '/learning/serverless-llm-apps-amazon-bedrock'

In [14]:
session = boto3.Session(profile_name="default")
bedrock = session.client('bedrock', region_name="eu-central-1")

In [15]:
from helpers.CloudWatchHelper import CloudWatch_Helper
cloudwatch = CloudWatch_Helper()

In [17]:
loggingConfig = {
    'cloudWatchConfig': {
        'logGroupName': log_group_name,
        'roleArn': role_arn,
        'largeDataDeliveryS3Config': {
            'bucketName': bucket_name,
            'keyPrefix': project_name + '/amazon_bedrock_large_data_delivery',
        }
    },
    's3Config': {
        'bucketName': bucket_name,
        'keyPrefix': 'amazon_bedrock_logs',
    },
    'textDataDeliveryEnabled': True
}

In [18]:
bedrock.put_model_invocation_logging_configuration(loggingConfig=loggingConfig)

{'ResponseMetadata': {'RequestId': 'cc23e8ab-caac-401a-a8ba-3c480a877034',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 01 May 2024 13:49:35 GMT',
   'content-type': 'application/json',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'cc23e8ab-caac-401a-a8ba-3c480a877034'},
  'RetryAttempts': 0}}

In [19]:
bedrock.get_model_invocation_logging_configuration()

{'ResponseMetadata': {'RequestId': '93779b1d-e63b-46e6-9f1d-fcde44406db6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 01 May 2024 13:49:43 GMT',
   'content-type': 'application/json',
   'content-length': '510',
   'connection': 'keep-alive',
   'x-amzn-requestid': '93779b1d-e63b-46e6-9f1d-fcde44406db6'},
  'RetryAttempts': 0},
 'loggingConfig': {'cloudWatchConfig': {'logGroupName': '/learning/serverless-llm-apps-amazon-bedrock',
   'roleArn': 'arn:aws:iam::165819210796:role/serverless-llm-apps-bedrock-role',
   'largeDataDeliveryS3Config': {'bucketName': 'learning-apps-bucket',
    'keyPrefix': 'serverless-llm-apps-amazon-bedrock/amazon_bedrock_large_data_delivery'}},
  's3Config': {'bucketName': 'learning-apps-bucket',
   'keyPrefix': 'amazon_bedrock_logs'},
  'textDataDeliveryEnabled': True,
  'imageDataDeliveryEnabled': True,
  'embeddingDataDeliveryEnabled': True}}

In [20]:
bedrock_runtime = session.client('bedrock-runtime', region_name="eu-central-1")

In [21]:
prompt = "Write an article about the fictional planet Foobar."

kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0.7,
                "topP": 0.9
            }
        }
    )
}

response = bedrock_runtime.invoke_model(**kwargs)
response_body = json.loads(response.get('body').read())

generation = response_body['results'][0]['outputText']

print(generation)


Foobar is a fictional planet in the popular science fiction series "Star Trek". It is a planet that is home to a species of humanoids known as the Foobarians. The Foobarians are a peaceful and technologically advanced race, known for their love of music and art.

The planet Foobar is located in the Beta Quadrant of the Milky Way Galaxy, and is approximately 10 light-years away from Earth. It is a lush, green planet with a breathable atmosphere and a wide variety of flora and fauna. The planet is home to a large population of Foobarians, who live in a series of cities and villages.

The Foobarians are a technologically advanced race, with a sophisticated society that is focused on art, music, and science. They have developed a form of transportation that allows them to travel at speeds faster than light, and have a powerful military force that is used solely for defense.

One of the most notable features of Foobar is the planet's unique musical culture. The Foobarians are known for the

In [23]:
cloudwatch.print_recent_logs(log_group_name)